In [1]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader,Conv2D
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [3]:
class Net(Module):
    def __init__(self, outc, impl):
        super().__init__()
        self.seq = Sequential(
            Conv2D(None, 64, 3, conv_impl=impl),
            ReLU(),
            lambda x: x.reshape(x.shape[0], -1),
            Linear(None, outc,bias=False)
        )

    def forward(self, x):
        x = self.seq(x)
        x = x.reshape(x.shape[0],-1)
        return x


def train(net, optim, train_X, train_y, epochs, batch_size=128):
    if train_X.ndim == 2:
        h=w=int(np.sqrt(train_X.shape[1]))
        train_X = train_X.reshape(train_X.shape[0],1,h,w)
    import time,tqdm,math
    start = time.monotonic()
    for epoch in range(epochs):  # loop over the dataset multiple times
        losses = []
        accs = []
        for X, y in (bar := tqdm.tqdm(dataloader(batch_size, train_X, train_y), total=math.ceil(len(train_y)/batch_size))):
            # forward + backward + optimize
            outputs = net.forward(X)
            loss = cross_entropy(outputs, y)
            optim.zero_grad()
            loss.backward()
            assert loss.item() >= 0, loss
            optim.step()

            losses.append(loss.item())
            accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
            bar.set_description(f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):3.3f} accuracy: {np.mean(accs)*100:3.2f}')
    return time.monotonic() - start

In [4]:
net = Net(np.unique(train_y).size, impl="slow")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch  10] loss: 0.189 accuracy: 99.17: 100%|██████████| 15/15 [00:01<00:00, 11.01it/s]

Finished Training in : 13.7350 s


In [5]:
net = Net(np.unique(train_y).size, impl="fast_forward")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch  10] loss: 0.314 accuracy: 99.01: 100%|██████████| 15/15 [00:01<00:00, 14.47it/s]

Finished Training in : 10.4690 s


In [6]:
net = Net(np.unique(train_y).size, impl="fast")
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer, train_X, train_y, 10)
print(f'Finished Training in : {time:.4f} s')

[Epoch  10] loss: 0.181 accuracy: 99.17: 100%|██████████| 15/15 [00:00<00:00, 106.53it/s]

Finished Training in : 3.5160 s
